In [3]:
example = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533"""

In [4]:
with open('./data/Day 17/input.txt') as f:
    data = f.read()

In [96]:
def parse_input(data):
    return [list(map(int, line)) for line in data.splitlines()]

# Part 1

In [185]:
##### CODE takes to long

from collections import deque
import numpy as np
def walk_grid(grid, start, end):
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    min_heatloss = float('inf')
    
    heatloss_grid_steps = [[[float('inf') for k in range(4)]for i in range(len(grid[0]))] for j in range(len(grid))]
    heatloss_grid = np.full((len(grid), len(grid[0])), float('inf'))

    queue = deque([(start, (1, 0), 0, 0), (start, (0, -1), 0, 0)])
    counter = 0
    while queue:
        counter += 1
        loc, prev_dir, steps_taken, heatloss = queue.pop()
        x, y = loc
        # Kill route if heatloss is larger than the smallest heatloss found so far
        if heatloss >= min_heatloss:
            continue

        # Kill route if another route has already found a lower heatloss for this location
        if heatloss >= heatloss_grid_steps[y][x][steps_taken]:
            continue

        # Kill route if we are out of bounds
        if x < 0 or y < 0 or x >= len(grid[0]) or y >= len(grid):
            continue

        # Kill route if we are at the goal
        if (x, y) == end:
            min_heatloss = min(min_heatloss, heatloss)
            heatloss_grid[y][x] = min(heatloss_grid[y][x], heatloss)
            heatloss_grid_steps[y][x][steps_taken] = min(heatloss_grid_steps[y][x][steps_taken], heatloss)
            continue

        # Knowing that heatloss 2091 is too high due to earlier greedy answer
        if heatloss >= 2091:
            continue

        # Add all possible directions to the queue
        for direction in directions:
            # Kill route if we are going back the way we came
            if direction == (-prev_dir[0], -prev_dir[1]):
                continue

            # Kill route if we are going the same direction as before and have taken more than 3 steps
            if direction == prev_dir:
                if steps_taken >= 3:
                    continue
                new_steps_taken = steps_taken + 1
            else:
                new_steps_taken = 1
            
            dx, dy = direction
            
            # Kill route if we are out of bounds
            if x + dx < 0 or y +dy < 0 or x + dx >= len(grid[0]) or y + dy >= len(grid):
                continue

            queue.append(((x + dx, y + dy), direction, new_steps_taken, heatloss + grid[y][x]))
            heatloss_grid_steps[y][x][steps_taken] = min(heatloss_grid_steps[y][x][steps_taken], heatloss)
            heatloss_grid[y][x] = min(heatloss, heatloss_grid[y][x])

        if counter % 1_000_000 == 0:
            print(f"Counter: {counter}, heatloss_grid_empty: {np.count_nonzero(np.where(heatloss_grid == float('inf'))[0])}, Current min heatloss: {min_heatloss}, Queue length: {len(queue)}")
    return min_heatloss, heatloss_grid, heatloss_grid_steps

In [178]:
example_data = parse_input(example)
min_heatloss, min_heatloss_grid, heatloss_grid_steps = walk_grid(example_data, (0, 0), (len(example_data[0])-1, len(example_data)-1))
print(min_heatloss)

102


In [210]:
### With help of reddit to learn that heapq is a thing

import heapq

def walk_grid(grid, end):
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    
    # locations, direction_index, steps_taken
    visited = set([((1, 0), 0, 1), ((0, 1), 2, 1)])

    # Heatloss, location, direction_index, steps_taken
    head = [(int(grid[0][1]), (1, 0), 0, 1), (int(grid[1][0]), (0, 1), 2, 1)]
    
    while head:
        heatloss, loc, direction_index, steps_taken,  = heapq.heappop(head)
        x, y = loc
        # Kill route if we are at the goal
        if (x, y) == end:
            return heatloss

        # Add all possible directions to the queue
        for i, new_direction in enumerate(directions):
            dx, dy = new_direction
            new_x, new_y = x + dx, y + dy
            # Kill route if we are out of bounds
            if new_x < 0 or new_y < 0 or new_x >= len(grid[0]) or new_y >= len(grid) or (i == direction_index and steps_taken >= 3):
                continue

            # Kill route if we are going back the way we came
            if new_direction == (-directions[direction_index][0], -directions[direction_index][1]):
                continue

            new_heatloss = heatloss + grid[new_y][new_x]
            new_steps_taken = steps_taken + 1 if i == direction_index else 1

            if ((new_x, new_y), i, new_steps_taken) not in visited:
                heapq.heappush(head, (new_heatloss, (new_x, new_y), i, new_steps_taken))
                visited.add(((new_x, new_y), i, new_steps_taken))

In [213]:
example_data = parse_input(example)
min_heatloss = walk_grid(example_data, (len(example_data[0])-1, len(example_data)-1))
print(min_heatloss)

102


In [214]:
heatmap_parsed = parse_input(data)
min_heatloss = walk_grid(heatmap_parsed, (len(heatmap_parsed[0])-1, len(heatmap_parsed)-1))
print(min_heatloss)

847


# Part 2

In [219]:

import heapq

def walk_grid_distances(grid, end, min_count = 4, max_count = 10):
    directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    
    # locations, direction_index, steps_taken
    visited = set([((1, 0), 0, 1), ((0, 1), 2, 1)])

    # Heatloss, location, direction_index, steps_taken
    head = [(int(grid[0][1]), (1, 0), 0, 1), (int(grid[1][0]), (0, 1), 2, 1)]
    
    while head:
        heatloss, loc, direction_index, steps_taken,  = heapq.heappop(head)
        x, y = loc
        # Kill route if we are at the goal
        if (x, y) == end and steps_taken >= min_count and steps_taken <= max_count:
            return heatloss
        
        # Add all possible directions to the queue
        for i, new_direction in enumerate(directions):
            dx, dy = new_direction
            new_x, new_y = x + dx, y + dy
            # Kill route if we are out of bounds
            if new_x < 0 or new_y < 0 or new_x >= len(grid[0]) or new_y >= len(grid) or (i == direction_index and steps_taken >= max_count):
                continue

            # Kill route if we are going back the way we came
            if new_direction == (-directions[direction_index][0], -directions[direction_index][1]):
                continue

            if i != direction_index and steps_taken < min_count:
                continue

            new_heatloss = heatloss + grid[new_y][new_x]
            new_steps_taken = steps_taken + 1 if i == direction_index else 1

            if ((new_x, new_y), i, new_steps_taken) not in visited:
                heapq.heappush(head, (new_heatloss, (new_x, new_y), i, new_steps_taken))
                visited.add(((new_x, new_y), i, new_steps_taken))

In [220]:
example_data = parse_input(example)
min_heatloss = walk_grid_distances(example_data, (len(example_data[0])-1, len(example_data)-1))
print(min_heatloss)

94


In [221]:
heatmap_parsed = parse_input(data)
min_heatloss = walk_grid_distances(heatmap_parsed, (len(heatmap_parsed[0])-1, len(heatmap_parsed)-1))
print(min_heatloss)

997
